# Hybrid Search using RRF

In this example we'll use the reciprocal rank fusion algorithm to combine the results of BM25 and kNN semantic search.
We'll use the same dataset we used in our [quickstart](https://github.com/joemcelroy/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/00-quick-start.ipynb) guide.

# 🧰 Requirements

For this example, you will need:

- Python 3.6 or later
- An Elastic deployment with minimum **4GB machine learning node**
   - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?elektra=en-ess-sign-up-page))
- The [ELSER](https://www.elastic.co/guide/en/machine-learning/8.8/ml-nlp-elser.html) model installed on your Elastic deployment
- The [Elastic Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/installation.html)


# Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?fromURI=%2Fhome) for a free trial.

- Go to the [Create deployment](https://cloud.elastic.co/deployments/create) page
   - Under **Advanced settings**, go to **Machine Learning instances**
   - You'll need at least **4GB** RAM per zone for this tutorial
   - Select **Create deployment**

# Install packages and initialize the Elasticsearch Python client

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the packages we need for this example.

In [1]:
!git clone https://github.com/elastic/elasticsearch-py.git
%cd elasticsearch-py
!git checkout v8.8.2
!{sys.executable} -m pip install .
!pip install sentence_transformers
!pip install torch


fatal: destination path 'elasticsearch-py' already exists and is not an empty directory.
/Users/liamthompson/notebook-tests/elasticsearch-py
HEAD is now at 825e642b Bumps 8.8 to 8.8.2
zsh:1: parse error near `-m'
You should consider upgrading via the '/Users/liamthompson/.pyenv/versions/3.9.7/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/liamthompson/.pyenv/versions/3.9.7/bin/python3.9 -m pip install --upgrade pip' command.


[TODO: Update]
Next we need to import the `elasticsearch` module and the `getpass` module.
`getpass` is part of the Python standard library and is used to securely prompt for credentials.

In [2]:
from elasticsearch import Elasticsearch, helpers
from urllib.request import urlopen
import getpass
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

/Users/liamthompson/.pyenv/versions/3.9.7/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we can instantiate the Python Elasticsearch client.
First we prompt the user for their password and Cloud ID.

🔐 NOTE: `getpass` enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [68]:
# Found in the 'Manage Deployment' page
CLOUD_ID = getpass.getpass('Enter Elastic Cloud ID:  ')

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass('Enter Elastic password:  ')

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

Confirm that the client has connected with this test

In [69]:
print(client.info())

{'name': 'instance-0000000000', 'cluster_name': '9dd1e5c0b0d64796b8cf0746cf63d734', 'cluster_uuid': 'VeYvw6JhQcC3P-Q1-L9P_w', 'version': {'number': '8.9.0-SNAPSHOT', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'ac7d79178c3e57c935358453331efe9e9cc5104d', 'build_date': '2023-06-21T09:08:25.219504984Z', 'build_snapshot': True, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0', 'transport_version': '8500019'}, 'tagline': 'You Know, for Search'}


Refer to https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect to a self-managed deployment.

Read https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect using API keys.


# Create Elasticsearch index with required mappings

We need to add a field to support dense vector storage and search.
Note the `title_vector` field below, which is used to store the dense vector representation of the `title` field.

In [59]:
# Define the mapping
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "authors": {"type": "keyword"},
            "summary": {"type": "text"},
            "publish_date": {"type": "date"},
            "num_reviews": {"type": "integer"},
            "publisher": {"type": "keyword"},
            "title_vector": { 
                "type": "dense_vector", 
                "dims": 384, 
                "index": "true", 
                "similarity": "dot_product" 
            }
        }
    }
}

# Create the index
client.indices.create(index='rrf_book_index', body=mapping)


/var/folders/vz/v2f6_x6s0kg51j2vbm5rlhww0000gn/T/ipykernel_2383/1628078329.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.indices.create(index='rrf_book_index', body=mapping)


BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [rrf_book_index/Ip8zitwhSMe0OJtEwpuqzQ] already exists')

## Dataset

Let's index some data.
Note that we are embedding the `title` field using the sentence transformer model.
Once indexed, you'll see that your documents contain a `title_vector` field (`"type": "dense_vector"`) which contains a vector of floating point values.
This is the embedding of the `title` field in vector space.
We'll use this field to perform semantic search using kNN.

In [17]:
books = [
    {
        "title": "The Pragmatic Programmer: Your Journey to Mastery",
        "authors": ["andrew hunt", "david thomas"],
        "summary": "A guide to pragmatic programming for software engineers and developers",
        "publish_date": "2019-10-29",
        "num_reviews": 30,
        "publisher": "addison-wesley"
    },
    {
        "title": "Python Crash Course",
        "authors": ["eric matthes"],
        "summary": "A fast-paced, no-nonsense guide to programming in Python",
        "publish_date": "2019-05-03",
        "num_reviews": 42,
        "publisher": "no starch press"
    },
    {
        "title": "Artificial Intelligence: A Modern Approach",
        "authors": ["stuart russell", "peter norvig"],
        "summary": "Comprehensive introduction to the theory and practice of artificial intelligence",
        "publish_date": "2020-04-06",
        "num_reviews": 39,
        "publisher": "pearson"
    },
    {
        "title": "Clean Code: A Handbook of Agile Software Craftsmanship",
        "authors": ["robert c. martin"],
        "summary": "A guide to writing code that is easy to read, understand and maintain",
        "publish_date": "2008-08-11",
        "num_reviews": 55,
        "publisher": "prentice hall"
    },
    {
        "title": "You Don't Know JS: Up & Going",
        "authors": ["kyle simpson"],
        "summary": "Introduction to JavaScript and programming as a whole",
        "publish_date": "2015-03-27",
        "num_reviews": 36,
        "publisher": "oreilly"
    },
    {
        "title": "Eloquent JavaScript",
        "authors": ["marijn haverbeke"],
        "summary": "A modern introduction to programming",
        "publish_date": "2018-12-04",
        "num_reviews": 38,
        "publisher": "no starch press"
    },
    {
        "title": "Design Patterns: Elements of Reusable Object-Oriented Software",
        "authors": ["erich gamma", "richard helm", "ralph johnson", "john vlissides"],
        "summary": "Guide to design patterns that can be used in any object-oriented language",
        "publish_date": "1994-10-31",
        "num_reviews": 45,
        "publisher": "addison-wesley"
    },
    {
        "title": "The Clean Coder: A Code of Conduct for Professional Programmers",
        "authors": ["robert c. martin"],
        "summary": "A guide to professional conduct in the field of software engineering",
        "publish_date": "2011-05-13",
        "num_reviews": 20,
        "publisher": "prentice hall"
    },
    {
        "title": "JavaScript: The Good Parts",
        "authors": ["douglas crockford"],
        "summary": "A deep dive into the parts of JavaScript that are essential to writing maintainable code",
        "publish_date": "2008-05-15",
        "num_reviews": 51,
        "publisher": "oreilly"
    },
    {
        "title": "Introduction to the Theory of Computation",
        "authors": ["michael sipser"],
        "summary": "Introduction to the theory of computation and complexity theory",
        "publish_date": "2012-06-27",
        "num_reviews": 33,
        "publisher": "cengage learning"
    },
]

ObjectApiResponse({'took': 29, 'errors': False, 'items': [{'index': {'_index': 'rrf_book_index', '_id': '7c-QKokBaD3r4jKCZkdN', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 10, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': '7s-QKokBaD3r4jKCZkdN', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 11, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': '78-QKokBaD3r4jKCZkdN', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 12, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': '8M-QKokBaD3r4jKCZkdN', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 13, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': '8c-QKokBaD3r4jKCZkdN', '_version': 1, 'resu

## Index documents

Our dataset is a Python list that contains dictionaries of movie titles and descriptions.
We'll use the `helpers.bulk` method to index our documents in batches.

The following code iterates over the list of books and creates a list of actions to be performed.
Each action is a dictionary containing an "index" operation on our Elasticsearch index.
The book's title is encoded using our selected model, and the encoded vector is added to the book document.
The book document is then added to the list of actions.

Finally, we call the `bulk` method, specifying the index name and the list of actions.

In [70]:
actions = []
for book in books:
    actions.append({"index": {"_index": "rrf_book_index"}})
    titleEmbedding = model.encode(book["title"]).tolist()
    book["title_vector"] = titleEmbedding
    actions.append(book)

client.bulk(index="rrf_book_index", operations=actions)

ObjectApiResponse({'took': 25, 'errors': False, 'items': [{'index': {'_index': 'rrf_book_index', '_id': 'KM-gK4kBaD3r4jKC2Ejk', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 30, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'Kc-gK4kBaD3r4jKC2Ejk', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 31, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'Ks-gK4kBaD3r4jKC2Ejk', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 32, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'K8-gK4kBaD3r4jKC2Ejk', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 33, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'LM-gK4kBaD3r4jKC2Ejk', '_version': 1, 'resu

## Pretty printing Elasticsearch responses

This is a helper function to print Elasticsearch responses in a readable format.

In [71]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        publication_date = hit['_source']['publish_date']
        score = hit['_score']
        title = hit['_source']['title']
        summary = hit['_source']['summary']
        pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nScore: {score}")
        print(pretty_output)

# Hybrid search using RRF

## RRF overview

[Reciprocal Rank Fusion (RRF)](https://www.elastic.co/guide/en/elasticsearch/reference/current/rrf.html) is a state-of-the-art ranking algorithm for combining results from different information retrieval strategies.
RRF consistently improves the combined results of different search algorithms.
It outperforms all other ranking algorithms, and often surpasses the best individual results, without calibration.
In brief, it enables best-in-class hybrid search out of the box.

## How RRF works in Elasticsearch

You can use RRF as part of a search to combine and rank documents using result sets from a combination of query and/or knn searches.
A minimum of 2 results sets is required for ranking from the specified sources.
Check out the [RRF API reference](https://www.elastic.co/guide/en/elasticsearch/reference/master/rrf.html#rrf-api) for full details information.

In the following example, we'll use RRF to combine the results of a `match` query and a kNN semantic search.


In [76]:
body = {
    "size": 5,
    "query": {
        "match": {
            "summary": "shoes"
        },
        
    },
    "knn": {
        "field": "title_vector",
        "query_vector" : model.encode("python programming").tolist(), # generate embedding for query so it can be compared to `title_vector`
        "k": 5,
        "num_candidates": 10},
    "rank": {
        "rrf": {
            "window_size": 5,
            "rank_constant": 20
        }
    }
}

response = client.search(index="rrf_book_index", body=body)

print(response)

/var/folders/vz/v2f6_x6s0kg51j2vbm5rlhww0000gn/T/ipykernel_2383/2934485565.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="rrf_book_index", body=body)


TypeError: search() got an unexpected keyword argument 'rank'

In the above example, we first execute the kNN search to get its global top 5 results.
Then we execute the match query to get its global top 5 results.
Then we combine the knn search and match query results and rank them based on the RRF method to get the final top 2 results.

ℹ️ Note that if `k` from a knn search is larger than `window_size`, the results are truncated to `window_size`.
If `k` is smaller than `window_size`, the results will be `k` size.

## RRF toy example

This very simple example demonstrates how RRF ranks documents from different search strategies.
We begin by creating a mapping for an index with a text field, a vector field, and an integer field along with indexing several documents. For this example we are going to use a vector with only a single dimension to make the ranking easier to explain.

In [ ]:
body = {
  "mappings": {
        "properties": {
            "text" : {
                "type" : "text"
            },
            "vector": {
                "type": "dense_vector",
                "dims": 1,
                "similarity": "l2_norm",
                "index": "true"

            },
            "integer" : {
                "type" : "integer"
            }
        }
    }
}

client.indices.create(index="example-index", body=body)

Next let's index some documents.

In [80]:
doc1 = {
    "text" : "rrf",
    "vector" : [5],
    "integer": 1
}

doc2 ={
    "text" : "rrf rrf",
    "vector" : [4],
    "integer": 2
}

doc3 = {
    "text" : "rrf rrf rrf",
    "vector" : [3],
    "integer": 1
}

doc4 = {
    "text" : "rrf rrf rrf rrf",
    "integer": 2
}

doc5 ={
    "vector" : [0],
    "integer": 1
}

docs = [doc1, doc2, doc3, doc4, doc5]

actions = []
for doc in docs:
    actions.append({"index": {"_index": "example-index"}})
    actions.append(doc)

client.bulk(index="example-index", operations=actions)

ObjectApiResponse({'took': 7, 'errors': False, 'items': [{'index': {'_index': 'example-index', '_id': 'UM8cLYkBaD3r4jKCTUjQ', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'example-index', '_id': 'Uc8cLYkBaD3r4jKCTUjQ', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'example-index', '_id': 'Us8cLYkBaD3r4jKCTUjQ', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'example-index', '_id': 'U88cLYkBaD3r4jKCTUjQ', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'example-index', '_id': 'VM8cLYkBaD3r4jKCTUjQ', '_version': 1, 'result': 'crea

We now execute a search using RRF with a query, a kNN search, and a terms aggregation.

In [81]:
body = {
    "query": {
        "term": {
            "text": "rrf"
        }
    },
    "knn": {
        "field": "vector",
        "query_vector": [3],
        "k": 5,
        "num_candidates": 5
    },
    "rank": {
        "rrf": {
            "window_size": 5,
            "rank_constant": 1
        }
    },
    "size": 3,
    "aggs": {
        "int_count": {
            "terms": {
                "field": "integer"
            }
        }
    }
}

response = client.search(index="example-index", body=body)

/var/folders/vz/v2f6_x6s0kg51j2vbm5rlhww0000gn/T/ipykernel_2383/3671365121.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="example-index", body=body)


TypeError: search() got an unexpected keyword argument 'rank'

We receive a response with ranked hits and the terms aggregation result.
Note that _score is null, and we instead use _rank to show our top-ranked documents.

Let’s break down how these hits were ranked.
We start by running the query and the kNN search separately to collect what their individual hits are.

First, we look at the hits for the query.

```json
"hits" : [
    {
        "_index" : "example-index",
        "_id" : "4",
        "_score" : 0.16152832, (1)               
        "_source" : {
            "integer" : 2,
            "text" : "rrf rrf rrf rrf"
        }
    },
    {
        "_index" : "example-index",
        "_id" : "3",                    (2)    
        "_score" : 0.15876243,
        "_source" : {
            "integer" : 1,
            "vector" : [3],
            "text" : "rrf rrf rrf"
        }
    },
    {
        "_index" : "example-index",
        "_id" : "2",                       (3) 
        "_score" : 0.15350538,
        "_source" : {
            "integer" : 2,
            "vector" : [4],
            "text" : "rrf rrf"
        }
    },
    {
        "_index" : "example-index",
        "_id" : "1",                        (4)
        "_score" : 0.13963442,
        "_source" : {
            "integer" : 1,
            "vector" : [5],
            "text" : "rrf"
        }
    }
]
```

```markdown
<1> rank 1, `_id` 4
<2> rank 2, `_id` 3
<3> rank 3, `_id` 2
<4> rank 4, `_id` 1
```


Note that our first hit doesn’t have a value for the vector field.

Now, we look at the results for the kNN search.

```json
"hits" : [
    {
        "_index" : "example-index",
        "_id" : "3",                   
        "_score" : 1.0,
        "_source" : {
            "integer" : 1,
            "vector" : [3],
            "text" : "rrf rrf rrf"
        }
    },
    {
        "_index" : "example-index",
        "_id" : "2",                   
        "_score" : 0.5,
        "_source" : {
            "integer" : 2,
            "vector" : [4],
            "text" : "rrf rrf"
        }
    },
    {
        "_index" : "example-index",
        "_id" : "1",                   
        "_score" : 0.2,
        "_source" : {
            "integer" : 1,
            "vector" : [5],
            "text" : "rrf"
        }
    },
    {
        "_index" : "example-index",
        "_id" : "5",                   
        "_score" : 0.1,
        "_source" : {
            "integer" : 1,
            "vector" : [0]
        }
    }
]```

```markdown
<1> rank 1, `_id` 3
<2> rank 2, `_id` 2
<3> rank 3, `_id` 1
<4> rank 4, `_id` 5
```

We can now take the two individually ranked result sets and apply the RRF formula to them to get our final ranking.

```python
# doc  | query     | knn       | score
_id: 1 = 1.0/(1+4) + 1.0/(1+3) = 0.4500
_id: 2 = 1.0/(1+3) + 1.0/(1+2) = 0.5833
_id: 3 = 1.0/(1+2) + 1.0/(1+1) = 0.8333
_id: 4 = 1.0/(1+1)             = 0.5000
_id: 5 =             1.0/(1+4) = 0.2000
```

We rank the documents based on the RRF formula with a `window_size` of `5`
truncating the bottom `2` docs in our RRF result set with a `size` of `3`.
We end with `_id: 3` as `_rank: 1`, `_id: 2` as `_rank: 2`, and
`_id: 4` as `_rank: 3`. This ranking matches the result set from the
original RRF search as expected.